<a href="https://colab.research.google.com/github/Ramubala/face-recognition/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# from google.colab import drive
# drive.mount('/content/drive')

In [34]:
# !rm -rf '/content/data' 
# !rm -rf '/content/annotations'

In [29]:
# !mkdir data
# !mkdir annotations
# !tar zxvf '/content/drive/MyDrive/datasets/originalPics.tar.gz' -C '/content/data/' ;
# !tar zxvf '/content/drive/MyDrive/datasets/FDDB-folds.tgz' -C '/content/annotations/';

In [32]:
# import glob
# file_list = [file for file in glob.glob(r'/content/annotations/FDDB-folds/*-ellipseList.txt',recursive=True)]
# train_files, test_files = file_list[:9],[file_list[9]]
# print(train_files, test_files)

In [100]:
#custom_train_dataset[2]

In [83]:
# def per_person_stats(source_txt: str):
#   stats_dict=dict()
#   with open(source_txt,'r') as f:
#     data = f.readlines()
#     for i in range(1, len(data)):
#       line = data[i].strip().split("\t")
#       stats_dict[line[0]] = int(line[1])
#   f.close()
#   return stats_dict

In [66]:
# def pick_train_pairs(_dict, num_per_sample, num_negative_faces_per_sample):
#   if num_per_sample!=3:
#     raise Exception("only 3 per sample supported by algorithm")
#   allfaces = list(_dict.keys())
#   data = []
#   for key in allfaces:
#     # anchor and postive example
#     facepath = '/content/data/lfw-py/lfw_funneled/{}'.format(key)
#     available_images = [file for file in glob.glob(facepath+'/*.jpg')]
#     num_available_images = _dict[key]
#     num_achor_postive_pairs = num_available_images*(num_available_images-1)
#     if num_achor_postive_pairs==0:
#       combinations = [(available_images[0],available_images[0])]
#     else:
#       combinations = list(itertools.combinations(available_images, 2))

#     # negative face
#     possible_choices = [v for v in allfaces if v != key]
#     negative_examples = random.sample(possible_choices, num_negative_faces_per_sample)
#     for negative_example in negative_examples:
#       negfacepath = '/content/data/lfw-py/lfw_funneled/{}'.format(negative_example)
#       available_images = [file for file in glob.glob(negfacepath+'/*.jpg')]
#       neg_imgpath = random.choice(available_images)

#       for tuple_el in combinations:
#         data.append((tuple_el[0],tuple_el[1],neg_imgpath))
#   return data  

In [102]:
# train_txt_file = '/content/data/lfw-py/peopleDevTrain.txt'
# test_txt_file = '/content/data/lfw-py/peopleDevTest.txt'
# train_stats = per_person_stats(train_txt_file)
# test_stats = per_person_stats(test_txt_file)

In [65]:
#test_stats.keys() & train_stats.keys()

In [103]:
# train_data = pick_train_pairs(train_stats, 3, 5)

In [105]:
import torchvision
import torch
import os
import glob
import numpy as np
import itertools
import random
random.seed(42)
from torch.utils.data import DataLoader,Dataset

In [70]:
train_dataset = torchvision.datasets.LFWPeople('/content/data/',split='train',download=True)
test_dataset = torchvision.datasets.LFWPeople('/content/data/',split='test',download=True)

Files already downloaded and verified
Files already downloaded and verified


In [73]:
len(train_dataset), len(test_dataset)

(9525, 3708)

In [81]:
train_dataset[0][0], train_dataset[0][1]

(<PIL.Image.Image image mode=RGB size=250x250 at 0x7F6AE9DEFC10>, 0)

In [101]:
class CustomDataset(Dataset):
  def __init__(self, src_dataset, upsampling_factor):
    self.src_dataset = src_dataset
    self.train_labels = np.array([src_dataset[i][1] for i in range(len(src_dataset))])
    self.labels = list(set(self.train_labels))
    self.label_to_index= {label: np.where(self.train_labels == label)[0]
                                     for label in self.labels}
    self.upsampling_factor = upsampling_factor
    
  def __len__(self):
      return self.upsampling_factor*len(self.src_dataset)

  def __getitem__(self, idx):
      # anchor image
      ref_dataset_index = int(idx/self.upsampling_factor)
      ref_train_image = self.src_dataset[ref_dataset_index]#[0], self.src_dataset[ref_dataset_index][1]
      label = ref_train_image[1]

      # finding positive example
      possible_ref_choices = self.label_to_index[label]
      choice = random.choice(possible_ref_choices)
      positive_train_image = self.src_dataset[choice]#[0], self.src_dataset[choice][1]

      # finding negative example
      possible_neg_choices = [v for v in self.labels if v!= label]
      choice = random.choice(possible_neg_choices)
      negative_train_image = self.src_dataset[choice]#[0], self.src_dataset[choice][1]

      return [ref_train_image, positive_train_image,negative_train_image]


_train_dataset = CustomDataset(train_dataset, 3)
_test_dataset = CustomDataset(test_dataset, 1)

In [104]:
len(_train_dataset), len(_test_dataset)

(28575, 3708)

In [106]:
torch.save(_train_dataset, 'train.pt')
torch.save(_test_dataset,'test.pt')